# COVID-19 Tracking Data Mart: Fact Table

In [64]:
import pandas as pd
import numpy as np

cases = pd.read_csv("conposcovidloc_20201027_20210227_ottawa_toronto.csv")

onset_date_dimension = pd.read_csv("Onset_Date_dimension.csv")
reported_date_dimension = pd.read_csv("Reported_Date_dimension.csv")
test_date_dimension = pd.read_csv("Test_Date_dimension.csv")
specimen_date_dimension = pd.read_csv("Specimen_Date_dimension.csv")
patient_dimension = pd.read_csv("Patient_dimension.csv")
PHU_dimension = pd.read_csv("PHU_dimension.csv")
mobility_dimension = pd.read_csv("mobility_dimension.csv")
special_measures_dimension = pd.read_csv("Special_Measures_Dimension.csv")
weather_dimension = pd.read_csv("weather_dimension.csv")

In [65]:
cases

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,85416,2020-10-23,2020-10-27,2020-10-27,2020-07-24,30s,FEMALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
1,16162,2020-10-23,2020-10-27,2020-10-27,2020-09-25,30s,FEMALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
2,143096,2020-10-21,2020-10-27,2020-10-27,2020-10-03,40s,FEMALE,CC,Resolved,No,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
3,83984,2020-10-27,2020-10-27,2020-10-27,2020-10-16,70s,FEMALE,OB,Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
4,179091,2020-10-19,2020-10-27,2020-10-27,2020-10-19,40s,FEMALE,MISSING INFORMATION,Resolved,No,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76451,232836,2021-02-26,2021-02-26,2021-03-02,2021-03-01,50s,FEMALE,CC,Not Resolved,No,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
76452,300702,2021-02-26,2021-02-26,2021-03-02,2021-03-01,40s,FEMALE,OB,Not Resolved,Yes,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
76453,204187,2021-02-19,2021-02-26,2021-03-05,2021-03-04,70s,FEMALE,CC,Resolved,No,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
76454,202101,2021-02-26,2021-02-26,NaN,2021-03-06,50s,FEMALE,CC,Not Resolved,No,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358


In [66]:
# function to test for missing values 

def count_missing(table, col):
    count = 0
    for idx, row in table.iterrows():
        if pd.isna(row[col]):
            count = count + 1
    print(col, count)
    

In [67]:
# checking for missing values in all columns

count_missing(cases, "Accurate_Episode_Date")
count_missing(cases, "Case_Reported_Date")
count_missing(cases, "Test_Reported_Date")
count_missing(cases, "Specimen_Date")
count_missing(cases, "Age_Group")
count_missing(cases, "Client_Gender")
count_missing(cases, "Case_AcquisitionInfo")
count_missing(cases, "Outcome1")
count_missing(cases, "Outbreak_Related")
count_missing(cases, "Reporting_PHU_ID")
count_missing(cases, "Reporting_PHU")
count_missing(cases, "Reporting_PHU_Address")
count_missing(cases, "Reporting_PHU_City")
count_missing(cases, "Reporting_PHU_Postal_Code")
count_missing(cases, "Reporting_PHU_Website")
count_missing(cases, "Reporting_PHU_Latitude")
count_missing(cases, "Reporting_PHU_Longitude")

Accurate_Episode_Date 0
Case_Reported_Date 0
Test_Reported_Date 7649
Specimen_Date 139
Age_Group 0
Client_Gender 0
Case_AcquisitionInfo 0
Outcome1 0
Outbreak_Related 0
Reporting_PHU_ID 0
Reporting_PHU 0
Reporting_PHU_Address 0
Reporting_PHU_City 0
Reporting_PHU_Postal_Code 0
Reporting_PHU_Website 0
Reporting_PHU_Latitude 0
Reporting_PHU_Longitude 0


In [68]:
# fixing missing values for Test and Specimen dates (if missing value, set to Case Reported date)

for idx, row in cases.iterrows():
    if pd.isna(row["Test_Reported_Date"]):
        cases.at[idx, "Test_Reported_Date"] = row["Case_Reported_Date"]   
    if pd.isna(row["Specimen_Date"]):
        cases.at[idx, "Specimen_Date"] = row["Case_Reported_Date"]

In [69]:
# checking all missing values fixed

count_missing(cases, "Test_Reported_Date")
count_missing(cases, "Specimen_Date")

Test_Reported_Date 0
Specimen_Date 0


In [70]:
# create Fact Table data frame

fact_table = pd.DataFrame(columns=['Onset_Date_Surrogate_Key', 'Reported_Date_Surrogate_Key', 'Test_Date_Surrogate_Key', 
                                     'Specimen_Date_Surrogate_Key', 'Patient_Surrogate_Key', 'PHU_Surrogate_Key', 
                                     'Mobility_Surrogate_Key', 'Special_Measures_Surrogate_Key', 'Weather_Surrogate_Key',
                                    'Resolved', 'Unresolved', 'Fatal'])


In [71]:
# fixing city values in mobility dimension for mapping to fact table

mobility_dimension_df =  pd.DataFrame(columns=['Mobility_Surrogate_Key','Mobility_Key', 'Date',
                                    'City', 'Province',"Retail_and_Recreation","Grocery_and_Pharmacy",
                                   'Parks', 'Transit_Stations', 'Workplaces', 'Residential'])

for idx, row in mobility_dimension.iterrows():
    m_row = [idx]
    m_row.append(row["Mobility_key"])
    m_row.append(row['Date'])
    if row['Subregion'] == "Ottawa Division":
        m_row.append("Ottawa")
    else:
        m_row.append("Toronto")
    m_row.append(row['Province'])
    m_row.append(row['Retail_and_recreation'])
    m_row.append(row['Grocery_and_phramacy'])
    m_row.append(row['Parks'])  
    m_row.append(row['Transit_stations']) 
    m_row.append(row['Workplaces'])
    m_row.append(row['Residential'])
    mobility_dimension_df.loc[len(mobility_dimension_df)] = m_row

mobility_dimension_df

,Mobility_Surrogate_Key,Mobility_Key,Date,City,Province,Retail_and_Recreation,Grocery_and_Pharmacy,Parks,Transit_Stations,Workplaces,Residential
0,0,0,2020-10-27,Ottawa,Ontario,-31.0,-5.0,-15.0,-63.0,-53.0,19.0
1,1,1,2020-10-28,Ottawa,Ontario,-31.0,-6.0,-9.0,-63.0,-52.0,19.0
2,2,2,2020-10-29,Ottawa,Ontario,-28.0,-2.0,-21.0,-63.0,-52.0,19.0
3,3,3,2020-10-30,Ottawa,Ontario,-32.0,-3.0,-2.0,-59.0,-48.0,19.0
4,4,4,2020-10-31,Ottawa,Ontario,-26.0,3.0,51.0,-47.0,-6.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...
243,243,243,2021-02-23,Toronto,Ontario,-53.0,-18.0,-24.0,-66.0,-52.0,21.0
244,244,244,2021-02-24,Toronto,Ontario,-53.0,-20.0,-20.0,-66.0,-52.0,22.0
245,245,245,2021-02-25,Toronto,Ontario,-52.0,-15.0,-11.0,-65.0,-51.0,22.0
246,246,246,2021-02-26,Toronto,Ontario,-50.0,-14.0,-6.0,-63.0,-49.0,21.0


In [72]:
special_measures_dimension

,Special_Measures_Surrogate_Key,City,Zone_Measures,Start_Date,End_Date,Description,Key_Word,Lockdown,Mandatory Masks,Indoor Dining,Gyms Open,Stay-at-home Order,Max Indoor Capacity,Max Outdoor Capacity
0,0,Ottawa,Orange,2020-10-27,2020-12-26,Ottawa is designated as a Restrict/Orange Zone...,Indoor services limited to 50 people,0,1,1,1,0,50,100
1,1,Ottawa,Grey,2020-12-26,2021-01-12,Province wide lockdown begins,Lockdown,1,1,0,0,1,0,10
2,2,Ottawa,Grey,2021-01-12,2021-02-08,Ontario extends province wide lockdown,Lockdown extended,1,1,0,0,1,0,10
3,3,Ottawa,Grey,2021-02-08,2021-02-16,Ottawa lockdown for another week,Lockdown extended,1,1,0,0,1,0,10
4,4,Ottawa,Orange,2021-02-16,2021-02-27,"Ottawa stay-at home order ends, back to being ...",Indoor services limited to 50 people,0,1,1,1,0,50,100
5,5,Toronto,Orange,2020-10-27,2020-11-14,Toronto is designated as a Restrict/Orange Zon...,Indoor services limited to 50 people,0,1,1,1,0,50,100
6,6,Toronto,Red,2020-11-14,2020-11-23,Toronto is declared a red zone(only outdoor di...,Indoor services limited to 10 people,0,1,1,1,0,10,25
7,7,Toronto,Grey,2020-11-23,2020-12-26,"Toronto goes into lockdown, designated as grey...",Lockdown,1,1,0,0,0,0,10
8,8,Toronto,Grey,2020-12-26,2021-01-12,Province wide lockdown begins,Lockdown extended,1,1,0,0,1,0,10
9,9,Toronto,Grey,2021-01-12,2021-02-19,Toronto continues to be in lockdown,Lockdown extended,1,1,0,0,1,0,10


In [73]:
# checking for error in special measures dimension

print(special_measures_dimension["City"].unique())

['Ottawa' 'Toronto']


In [74]:
# check unique values for regions in weather dimension

print(weather_dimension['Station Name'].unique())

['OTTAWA INTL A' 'OTTAWA CDA RCS' 'TORONTO INTL A' 'TORONTO CITY']


In [75]:
# fixing city values in weather dimension for mapping to fact table

weather_dimension_df =  pd.DataFrame(columns=['Weather_Surrogate_Key','City', 'Date',
                                    'Mean Temp (celsius)', 'Min Temp (celsius)','Max Temp (celsius)','Total Precip (mm)'])

for idx, row in weather_dimension.iterrows():
    w_row = [idx]
    if row['Station Name'] in ['OTTAWA INTL A', 'OTTAWA CDA RCS']:
        w_row.append("Ottawa")
    else:
        w_row.append("Toronto")
    w_row.append(row['Date/Time'])
    w_row.append(row['Mean Temp (°C)'])
    w_row.append(row['Min Temp (°C)'])
    w_row.append(row['Max Temp (°C)'])  
    w_row.append(row['Total Precip (mm)']) 
    weather_dimension_df.loc[len(weather_dimension_df)] = w_row

weather_dimension_df

,Weather_Surrogate_Key,City,Date,Mean Temp (celsius),Min Temp (celsius),Max Temp (celsius),Total Precip (mm)
0,0,Ottawa,2020-10-27,0.2,-2.3,2.7,0.0
1,1,Ottawa,2020-10-28,4.2,-1.1,9.4,0.4
2,2,Ottawa,2020-10-29,2.4,-1.3,6.0,0.0
3,3,Ottawa,2020-10-30,-0.5,-4.9,3.9,0.0
4,4,Ottawa,2020-10-31,-1.0,-8.4,6.5,0.0
...,...,...,...,...,...,...,...
241,241,Toronto,2021-02-22,0.6,-1.7,2.8,4.8
242,242,Toronto,2021-02-23,1.8,-0.7,4.3,0.2
243,243,Toronto,2021-02-24,3.9,-0.6,8.4,0.0
244,244,Toronto,2021-02-25,-1.4,-3.7,0.9,0.0


In [76]:
# mapping all dimension surrogate keys and measures to the fact table 


for idx, row in cases.iterrows():

    cases_row = [
        
        onset_date_dimension[onset_date_dimension["Date_Full_Format"]==row["Accurate_Episode_Date"]]["Onset_Date_Surrogate_Key"].values[0],
        reported_date_dimension[onset_date_dimension["Date_Full_Format"]==row["Case_Reported_Date"]]["Reported_Date_Surrogate_Key"].values[0],
        test_date_dimension[test_date_dimension["Date_Full_Format"]==row["Test_Reported_Date"]]["Test_Date_Surrogate_Key"].values[0],
        specimen_date_dimension[specimen_date_dimension["Date_Full_Format"]==row["Specimen_Date"]]["Specimen_Date_Surrogate_Key"].values[0],
                      
        patient_dimension[
            (patient_dimension["Age"]==row["Age_Group"])
            &(patient_dimension["Gender"]==row["Client_Gender"])
            &(patient_dimension["Acquisition_Group"]==row["Case_AcquisitionInfo"])
            &(patient_dimension["Outbreak_Related"]==row["Outbreak_Related"])]["Patient_Surrogate_Key"].values[0],
        
        PHU_dimension[
            (PHU_dimension["City"]==row["Reporting_PHU_City"])]["PHU_Surrogate_Key"].values[0],
    
        mobility_dimension_df[
            (mobility_dimension_df["Date"]==row["Case_Reported_Date"])
            &(mobility_dimension_df["City"]==row["Reporting_PHU_City"])]["Mobility_Surrogate_Key"].values[0],    
    
        special_measures_dimension[
            (special_measures_dimension["Start_Date"] <= row["Case_Reported_Date"])
            &(special_measures_dimension["End_Date"] > row["Case_Reported_Date"])
            &(special_measures_dimension["City"]==row["Reporting_PHU_City"])]["Special_Measures_Surrogate_Key"].values[0],

        weather_dimension_df[
            (weather_dimension_df["Date"]==row["Case_Reported_Date"])
            &(weather_dimension_df["City"]==row["Reporting_PHU_City"])]["Weather_Surrogate_Key"].values[0]]
        
    cases_row += [1 if row["Outcome1"] == "Resolved" else 0,
                  1 if row["Outcome1"] == "Not Resolved" else 0,
                  1 if row["Outcome1"] == "Fatal" else 0]
    
    fact_table.loc[len(fact_table)] = cases_row

In [77]:
fact_table

,Onset_Date_Surrogate_Key,Reported_Date_Surrogate_Key,Test_Date_Surrogate_Key,Specimen_Date_Surrogate_Key,Patient_Surrogate_Key,PHU_Surrogate_Key,Mobility_Surrogate_Key,Special_Measures_Surrogate_Key,Weather_Surrogate_Key,Resolved,Unresolved,Fatal
0,214,218,218,123,48,0,124,5,123,1,0,0
1,214,218,218,186,48,0,124,5,123,1,0,0
2,212,218,218,194,5,0,124,5,123,1,0,0
3,218,218,218,207,288,0,124,5,123,1,0,0
4,210,218,218,210,9,0,124,5,123,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
76451,340,340,344,343,101,0,246,10,245,0,1,0
76452,340,340,344,343,0,0,246,10,245,0,1,0
76453,333,340,347,346,293,0,246,10,245,1,0,0
76454,340,340,340,348,101,0,246,10,245,0,1,0


In [78]:
# Number of rows

fact_table.shape[0]

76456

In [79]:
# Checking for duplicates 

fact_table.drop_duplicates().shape[0]

56250

In [80]:
# Grouping by row type, size = count of duplicates

grouped_fact_table = fact_table.groupby(fact_table.columns.tolist(),as_index=False).size()

In [81]:
grouped_fact_table

,Onset_Date_Surrogate_Key,Reported_Date_Surrogate_Key,Test_Date_Surrogate_Key,Specimen_Date_Surrogate_Key,Patient_Surrogate_Key,PHU_Surrogate_Key,Mobility_Surrogate_Key,Special_Measures_Surrogate_Key,Weather_Surrogate_Key,Resolved,Unresolved,Fatal,size
0,0,275,275,106,0,0,181,7,180,1,0,0,1
1,12,239,239,12,195,0,145,6,144,1,0,0,1
2,25,221,221,219,65,0,127,5,126,1,0,0,1
3,40,233,233,83,243,0,139,5,138,1,0,0,1
4,128,290,290,288,111,0,196,8,195,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56245,340,340,345,339,257,0,246,10,245,0,1,0,1
56246,340,340,347,338,2,0,246,10,245,0,1,0,1
56247,340,340,347,339,389,0,246,10,245,0,1,0,1
56248,340,340,348,340,401,0,246,10,245,1,0,0,1


In [82]:
for idx, row in grouped_fact_table.iterrows():
    if row['size'] > 1:
        if row['Resolved'] == 1:
            row['Resolved'] = row['size']
        elif row['Unresolved'] == 1:
            row['Unresolved'] = row['size']
        elif row['Fatal'] == 1:
            row['Fatal'] = row['size']

In [83]:
del grouped_fact_table['size']

In [84]:
grouped_fact_table

,Onset_Date_Surrogate_Key,Reported_Date_Surrogate_Key,Test_Date_Surrogate_Key,Specimen_Date_Surrogate_Key,Patient_Surrogate_Key,PHU_Surrogate_Key,Mobility_Surrogate_Key,Special_Measures_Surrogate_Key,Weather_Surrogate_Key,Resolved,Unresolved,Fatal
0,0,275,275,106,0,0,181,7,180,1,0,0
1,12,239,239,12,195,0,145,6,144,1,0,0
2,25,221,221,219,65,0,127,5,126,1,0,0
3,40,233,233,83,243,0,139,5,138,1,0,0
4,128,290,290,288,111,0,196,8,195,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
56245,340,340,345,339,257,0,246,10,245,0,1,0
56246,340,340,347,338,2,0,246,10,245,0,1,0
56247,340,340,347,339,389,0,246,10,245,0,1,0
56248,340,340,348,340,401,0,246,10,245,1,0,0


In [85]:
grouped_fact_table.to_csv("Final_Fact_Table.csv",index=False)

In [86]:
# export modified case data 

cases.to_csv("Cases_Final.csv",index=False)
#special_measures_dimension_df.to_csv("Special_Measures_Dimension_Final.csv",index=False)